# 累積和とその周辺

# はじめに

この文書では、累積和、及びその周辺のアルゴリズム・データ構造について解説を行う。

累積和は、その内容自体は非常に簡単なものだが、非常に様々な問題においてあちらこちらに顔を出すので、重要なものでもある。

まずは累積和の基本的な考え方、実際の構築の仕方と利用方法から話を始め、ちょうど累積和の逆ともいえる通称「いもす法」、座標圧縮との組み合わせ、二次元累積和などに話を広げていく予定となっている。

# 累積和の基本

## なぜ累積和を使うのか？

ある $N$ 個の整数の要素を持つ数列 $A: A_1, A_2, ... , A_N$ があるとする。例えば、具体的には次のようなものだ。

$
2, -5, 42, 9, -38, 24, 19, -57, 88, 67
$

こういった感じの数列中の、連続した一部の合計、つまり連続部分和を求めることを考える。具体的に、```sum_part()``` の形で関数を作成することにしよう。ややこしくならないように、0-indexedの半開放で定義しておく。すなわち、```sum_part(a, b)``` のとき、返ってくる値は部分和 $A_{a-1} + A_{a} + ... + A_{b-2} + A_{b-2}$ とする。

また、あらかじめ $A$ はリストとして ```A``` に保存されているものとする。

In [1]:
A = [2, -5, 42, 9, -38, 24, 19, -57, 88, 67]

さて、単純に要素を足し合わせるだけなので、もちろん次のように書ける。

In [2]:
def sum_part(a, b):
    return sum(A[a:b])

print(sum_part(3, 6))  # = 9 - 38 + 24

-5


一回こっきり部分和を計算するだけならば、この方法でなんの問題もない。ただ、何度もおなじ計算を要求されるようなケースでは、関数ごとに計算量が $O(N)$ となるこの方法では、問題がある。

例えば、回答として $A_1, A_1+A_2, A_1+A_2+A_3 ... $ を全て求めよ、などと言われた場合はどうだろうか？上記のやり方だと計算量は $O(N^2)$ となり、$N \le 10^4$ あたりから間に合わなくなってくる。

……いや、でも、その場合は前から順番に足していって、その都度答えを出力すればいいだけじゃない？と思ったことだろう。  
まさに、それこそが累積和である。

## 累積和を求める

というわけで、さっそく累積和を使ってみよう。

累積和自体は広い概念を指す言葉だが、競技プログラミングにおいて累積和を用いるというとき、多くの場合は、連続部分和を計算するために事前計算を行うことを指す。つまり、最初に次のようなものを計算する。

$
[0, A_1, A_1+A_2, A_1+A_2+A_3, ... , \sum_{i=1}^{N}A_i]
$

最初に 0 を入れているのは、後で都合がいいからだ。さっそく計算させてみよう。

In [7]:
N = len(A)
cum_sum = [0] * (N+1)  # 累積和を保存していく

tmp = 0  # 現在の累積和を管理

for i in range(N):
    tmp += A[i]
    cum_sum[i+1] = tmp

print(cum_sum)

[0, 2, -3, 39, 48, 10, 34, 53, -4, 84, 151]


ちなみに、実は Python には累積和を自動的に求めてくれる ```itertools``` ライブラリの関数 ```accumulate``` が存在している。使用するためには、もちろんインポートする必要がある。

```itertools.accumulate``` は、キーワード引数 ```func``` で行う計算を柔軟に変更したり、```initial``` で初期値を設定したりもできる便利な関数だ。さっきの例なら、こう書ける。

In [6]:
from itertools import accumulate
N = len(A)
cum_sum = accumulate(A, initial=0)
print(cum_sum)
print(list(cum_sum))

[0, 2, -3, 39, 48, 10, 34, 53, -4, 84, 151]


見ての通り返ってくるのはイテレータなので、添字を利用するならリストに変換する必要がある。

こちらを使用するほうが便利なのだが、個人的には自分で書くことが多い。

さて、いずれにせよ、こうして得られた数列がなぜ便利なのかというと、連続部分和を素早く計算できるからだ。

例えば、$A_3+A_4+A_5$ を計算したいとする。これは、$A_1+A_2+A_3+A_4+A_5$ から、$A_1+A_2$ を引いたものに等しくなる。この計算を一般化することにより、あらゆる連続部分和を、二つの部分和の差から簡単に計算できるようになる。

というわけで、最初に書いた ```sum_part()``` を書き直してみよう。

In [9]:
csum_A = [0] * (N+1)
tmp = 0
for i in range(N):
    tmp += A[i]
    csum_A[i+1] = tmp

def sum_part(a, b):
    return csum_A[b] - csum_A[a]

sum_part(3, 6)

-5

これで、事前準備 $O(N)$ により、連続部分和を $O(1)$ で計算できるようになったのが分かると思う。

# いもす法

## いもす法とは

いもす法とは、ある一次元以上の空間における範囲操作を差分のみで簡便にあらわしていく手法で、累積和とは微分・積分のような関係に近い（と、個人的には思っている）。

その利点は、範囲に対する操作を $O(1)$ で行える点だが、その情報の取り出しには $O(N)$ を要する。

つまり、操作のクエリが大量にあり、最終的な状態を一回答えればいいような問題に有効な手法といえる。


いささか抽象的な説明だったが、少し具体的な例を考えてみよう。  
ある駐車場があり、各車の入庫と出庫の時刻が記録されているとする。駐車場が混雑していた、仮に10台以上が駐車していた時間はどれくらいの長さになるのかを知りたいとしたとき、どのようにすればいいだろうか？ただし、出庫時刻には車は駐車していないものとする。

まずは、敢えて単純な解き方から考えてみる。

時間軸を取るので、ひとまず ```parking``` というリストを作成することにしよう。```parking[t]``` は、時刻 $t$ に駐車場に止まっている車の台数とする。

各車ごとに入出庫の時間は分かっている。それぞれを $T_1, T_2$ とする。出庫の時刻にはいなくなっているので、$T_1$ ～ $T_2-1$ の時刻に対し、駐車場に止まっている台数に 1 を加えることになる。コードに起こすと、次のような感じだろうか。

In [ ]:
for t in range(t1, t2):
    parking[t] += 1

このコードの場合、問題となるのは、車 1台の情報を処理するのに $O(max(t))$ かかってしまうことだ。車の台数が少なければ問題ないが、多くなってくると間に合わなくなってしまう。

よって、解き方を変える必要がある。

ところで、我々が現実に時間軸に沿って駐車台数を確認していくときはどうするだろうか？いちいち毎時刻ごとに車を数え直したりはしないだろう（もちろん、時々は確認するだろうが）。  
その代わりに、さっき 3台駐車していたところに、2台入庫してきて、1台出庫したから現在は 4台……のように管理するのではないだろうか。つまり、以下のような感じである。

- 時刻 $a$ に 2台入庫
- 時刻 $b$ に 1台出庫
- 時刻 $c$ に 3台入庫し、1台出庫
- 以下同様

つまり、駐車台数の変化のみに注目するという訳である。

この手法の利点は、車 1台につき「入庫情報」と「出庫情報」のみ書き換えればいいため、$O(1)$ で処理ができるという点にある。

ところが、実際に時刻 $T$ における台数を確認するためには、差分情報を元に累積和を計算する必要がある。従って、$O(N)$ の計算量が必要となるわけだ。

## いもす法の実装

いもす法については、抽象的な説明が難しいので、具体的な問題を解きつつ、実装を見ていくことにする。

---
**問題**  
駐車場に、ある日のべ $N$ 台の車の出入りがあった。それぞれの車の入庫時刻と出庫時刻が与えられる。駐車場に $P$台以上の車が駐車していた時間を求めなさい。

ただし、開始時点での駐車台数は $0$ 台とし、入庫と出庫に掛かる時間については無視するものとする。

**入力:**
$
N P\\
B_1 E_1\\
B_2 E_2\\
...\\
B_N E_N\\
$
ただし $B_i, E_i$ は $i$台目の車の入庫時刻と出庫時刻である。また、入力は全て整数で行われる。

**制約**
$
1 \le N \le 10^6\\
1 \le B_i \le 10^6\\
1 \le E_i \le 10^6\\
$
各 $i$ $(1 \le i \le N)$ において $B_i \le E_i$

---

この問題を、いもす法を用いて順番に解いていくことを考えてみよう。

まずは入力の受け取りから。

In [ ]:
N, P  = map(int, input().split())
car_dat = [list(map(int, input().split())) for _ in range(N)]

いもす法のために、 ```parking_diff``` というリストを作成しよう。```parking_diff[t]``` は、時刻 $t$ における駐車台数の変化とする。もちろん初期値は全て 0 である。長さは、制約一杯とっておくことにする。

In [1]:
parking_diff = [0] * (10**6 + 1)